In [1]:
import decimal

In [2]:
decimal.getcontext()

Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

In [3]:
decimal.getcontext().prec = 14

In [4]:
decimal.getcontext()

Context(prec=14, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

In [5]:
decimal.getcontext().prec = 28

In [6]:
old_prec = decimal.getcontext().prec
decimal.getcontext().prec = 4
print(decimal.Decimal(1) / decimal.Decimal(3))
decimal.getcontext().prec = old_prec
print(decimal.Decimal(1) / decimal.Decimal(3))

0.3333
0.3333333333333333333333333333


In [7]:
class precision:
    def __init__(self, prec):
        self.prec = prec
        self.current_prec = decimal.getcontext().prec

    def __enter__(self):
        decimal.getcontext().prec = self.prec

    def __exit__(self, exc_type, exc_value, exc_tb):
        decimal.getcontext().prec = self.current_prec
        return False

In [8]:
with precision(3):
    print(decimal.Decimal(1) / decimal.Decimal(3))
print(decimal.Decimal(1) / decimal.Decimal(3))

0.333
0.3333333333333333333333333333


In [9]:
with decimal.localcontext() as ctx:
    ctx.prec = 3
    print(decimal.Decimal(1) / decimal.Decimal(3))
print(decimal.Decimal(1) / decimal.Decimal(3))

0.333
0.3333333333333333333333333333


In [10]:
from time import perf_counter, sleep

In [11]:
class Timer:
    def __init__(self):
        self.elapsed = 0

    def __enter__(self):
        self.start = perf_counter()
        return self

    def __exit__(self, exc_type, exc_value, exc_tb):
        self.stop = perf_counter()
        self.elapsed = self.stop - self.start
        return False

In [12]:
with Timer() as timer:
    sleep(1)
print(timer.elapsed)

1.000250200042501


In [13]:
print('hello')

hello


In [14]:
import sys

class OutToFile:
    def __init__(self, fname):
        self._fname = fname
        self._current_stdout = sys.stdout

    def __enter__(self):
        self._file = open(self._fname, 'w')
        sys.stdout = self._file

    def __exit__(self, exc_type, exc_value, exc_tb):
        sys.stdout = self._current_stdout
        self._file.close()
        return False

In [18]:
with OutToFile('test.txt'):
    print(sys.stdout)
    print('Line 1')
    print('Line 2')

In [19]:
print('Line 1')

Line 1


In [20]:
sys.stdout

In [21]:
with open('test.txt') as f:
    print(f.readlines())

["<_io.TextIOWrapper name='test.txt' mode='w' encoding='cp1254'>\n", 'Line 1\n', 'Line 2\n']


In [22]:
print('Line 1')

Line 1


In [23]:
class Tag:
    def __init__(self, tag):
        self._tag = tag

    def __enter__(self):
        print(f'<{self._tag}>', end='')

    def __exit__(self, exc_type, exc_value, exc_tb):
        print(f'</{self._tag}>', end='')
        return False

In [24]:
with Tag('p'):
    print('some ', end='')
    with Tag('b'):
        print('bold', end='')
    print(' text', end='')

<p>some <b>bold</b> text</p>

Title
- Items 1
  - sub item 1a
  - sub item 1b
- Items 2
  - sub item 2a
  - sub item 2b

In [27]:
class ListMaker:
    def __init__(self, title, prefix='- ', indent=3):
        self._title = title
        self._prefix = prefix
        self._indent = indent
        self._current_indent = 0
        print(title)

    def __enter__(self):
        self._current_indent += self._indent
        return self

    def __exit__(self, exc_type, exc_value, exc_tb):
        self._current_indent -= self._indent
        return False

    def print_item(self, arg):
        s = ' ' * self._current_indent + self._prefix + str(arg)
        print(s)

In [31]:
with ListMaker('Items') as lm:
    lm.print_item('Item 1')
    with lm:
        lm.print_item('sub item 1a')
        lm.print_item('sub item 1b')
        with lm:
            lm.print_item('******')
    lm.print_item('Item 2')
    with lm:
        lm.print_item('sub item 2a')
        lm.print_item('sub item 2b')

Items
   - Item 1
      - sub item 1a
      - sub item 1b
         - ******
   - Item 2
      - sub item 2a
      - sub item 2b


In [37]:
with OutToFile('mylist.txt'):
    with ListMaker('Items') as lm:
        lm.print_item('Item 1')
        with lm:
            lm.print_item('sub item 1a')
            lm.print_item('sub item 1b')
            with lm:
                lm.print_item('******')
        lm.print_item('Item 2')
        with lm:
            lm.print_item('sub item 2a')
            lm.print_item('sub item 2b')

In [39]:
with open('mylist.txt') as f:
    for row in f:
        print(row, end='')

Items
   - Item 1
      - sub item 1a
      - sub item 1b
         - ******
   - Item 2
      - sub item 2a
      - sub item 2b
